# Clasificación de Hojas de Café

## Cargar las anotaciones del dataset

In [16]:
import json

annotations_file="RoCoLe.json"

with open(annotations_file,'r') as f:
    annotations=json.load(f)

In [23]:
len(annotations)

1560

In [28]:
annotations[1]

{'ID': 'cjvnf878wfzzd0838svwmv6gu',
 'DataRow ID': 'cjvnehagg0jhv0ctne7uze0m7',
 'Labeled Data': 'https://storage.googleapis.com/labelbox-193903.appspot.com/cjs5g735orqbe0b29ip8a1k1s%2F24a51d91-3220-414f-479b-4182739c011c-C1P2E2.jpg',
 'Label': {'Leaf': [{'state': 'unhealthy',
    'geometry': [{'x': 105, 'y': 526},
     {'x': 213, 'y': 547},
     {'x': 357, 'y': 496},
     {'x': 518, 'y': 446},
     {'x': 718, 'y': 416},
     {'x': 911, 'y': 420},
     {'x': 1088, 'y': 420},
     {'x': 1185, 'y': 446},
     {'x': 1323, 'y': 471},
     {'x': 1396, 'y': 531},
     {'x': 1468, 'y': 586},
     {'x': 1539, 'y': 618},
     {'x': 1523, 'y': 632},
     {'x': 1488, 'y': 632},
     {'x': 1442, 'y': 643},
     {'x': 1415, 'y': 680},
     {'x': 1383, 'y': 719},
     {'x': 1341, 'y': 767},
     {'x': 1277, 'y': 804},
     {'x': 1180, 'y': 822},
     {'x': 1125, 'y': 864},
     {'x': 1026, 'y': 875},
     {'x': 895, 'y': 859},
     {'x': 794, 'y': 845},
     {'x': 672, 'y': 815},
     {'x': 610, 'y'

In [29]:
for annotation in annotations:
    #print(annotation["ID"])
    pass